<a href="https://colab.research.google.com/github/riszwinger/tensorflow_cert/blob/main/Useful_Links.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### [Epoch vs Batch Size vs Iteration](https://towardsdatascience.com/epoch-vs-iterations-vs-batch-size-4dfb9c7ce9c9)



We can divide the dataset of 2000 examples into batches of 500 then it will take 4 iterations to complete 1 epoch.


## [steps_per_epoch](https://datascience.stackexchange.com/questions/47405/what-to-set-in-steps-per-epoch-in-keras-fit-generator)

steps_per_epoch=Number of Iteration=num_of_rows_train_data // batch_size

You can set it equal to num_samples // batch_size, which is a typical choice.

However, steps_per_epoch give you the chance to "trick" the generator when updating the learning rate using ReduceLROnPlateau() callback, because this callback checks the drop of the loss once each epoch has finished. If the loss has stagnated for a patience number of consecutive epochs, the callback decreases the learning rate to "slow-cook" the network. If your dataset is huge, as it is usually the case when you need to use generators, you would probably like to decay the learning rate within a single epoch (since it includes a big number of data). This can be achieved by setting steps_per_epoch to a value that is less than num_samples // batch_size without affecting the overall number of training epochs of your model.

**validation_steps** similar to steps_per_epoch but on the validation data set instead on the training data. 

[steps_per_epoch deep dive](https://github.com/keras-team/keras/issues/10164)
*The steps_per_epoch argument is not related to how the samples are fed. It's just a number that is used to define an 'epoch'. Some people want their callbacks to be called more often than once per "real epoch"*. So they can set a lower steps_per_epoch.

The unseen samples will be seen in the second epoch
Your model will see the same samples multiple times.
*real epoch is a loop over all of your data*

## NLP

- [adapt](https://www.tensorflow.org/api_docs/python/tf/keras/layers/TextVectorization#adapt) after TextVectorization sets up the vocab which run only on the text data.
- The shape of input data needs to be rank-1 , shape=(1,) for the TextVectorization Layer.
- There is a performance difference to keep in mind when choosing where to apply your TextVectorization layer. Using it outside of your model enables you to do asynchronous CPU processing and buffering of your data when training on GPU. So, if you're training your model on the GPU, you probably want to go with this option to get the best performance while developing your model, then switch to including the TextVectorization layer inside your model when you're ready to prepare for deployment.
- **Don't use keywords like "train" in code.**

## Time Series

- [Basic](https://machinelearningmastery.com/decompose-time-series-data-trend-seasonality/)
- [Shape](https://shiva-verma.medium.com/understanding-input-and-output-shape-in-lstm-keras-c501ee95c65e)
- [RNN Shape](https://stackoverflow.com/questions/47268608/confusion-about-keras-rnn-input-shape-requirement)
- [CNN Shape](https://towardsdatascience.com/understanding-input-and-output-shapes-in-convolution-network-keras-f143923d56ca)

## Errors


### <u> Issue-1</u>
```history=model.fit(train_datagen,epochs=10,steps_per_epoch=STEP_PER_EPOCHS,validation_data=val_datagen,validation_steps=VALIDATION_SETPS)```


**InvalidArgumentError**:  logits and labels must have the same first dimension, got logits shape [32,3] and labels shape [96]
	 [[node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits (defined at <ipython-input-61-78e7226bd14d>:3) ]] [Op:__inference_train_function_1056]


**Solution:**

1. Looks like using wrong loss fnx in compile (SparseCategrocialCrossEntropy(from_logits=True) , may be changing this would help.
2. Checked the input data.
```
img,lbl=next(iter(train_datagen))
img[0].shape ##(180, 180, 3)
lbl[0] ##array([1., 0., 0.], dtype=float32)
```
3. Change the loss to CategoricalCrossentropy as below.
```
model.compile(optimizer='adam',loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
```

*Takeaway* : 
- sparse_categorical_crossentropy expects integer labels, as it does the one-hot encoding itself (hence, sparse).

- CategoricalCrossentropy : labels to be provided in a one_hot representation.


### <u> Issue-2</u>

*WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 945 batches). You may need to use the repeat() function when building your dataset.*

**Solution**

- Make sure to recacluate the values for Steps_per_epoch, and check the value to confirm


### <u> Issue-3</u>

TypeError: a bytes-like object is required, not 'dict'

```
train_txt[:2]
#[b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. ,
# b'I have been known to fall asl
#]
tokenObj.fit_on_texts(train_txt)
```
**Solution**

-  need to decode to make it like utf-8
```
tokenObj.fit_on_texts([i.decode('utf-8') for i in train_txt])
```



## GoodReads

- [medium link](https://towardsdatascience.com/nine-tools-i-wish-i-mastered-before-my-phd-in-machine-learning-708c6dcb2fb0)

## Must reads

- [Google Rules of ML](https://developers.google.com/machine-learning/guides/rules-of-ml)

